# 1. Imports

In [2]:
# local tests
import requests
import zipfile
import io
import os

# util functions
from src.utils.helper import get_soup, recursive_field_dict, recursive_find_value, flatten_dict, get_atom_data, get_main_df
from src.utils.mappings import mappings


source_url = 'https://www.hacienda.gob.es/es-ES/GobiernoAbierto/Datos%20Abiertos/Paginas/LicitacionesContratante.aspx'

# 2. Explore

In [3]:
# Get the soup of the source data url
soup = get_soup(source_url)

# Create a dict with keys the period of the data and the link to the data as values
links = [a.get('href') for a in soup.find_all('a') if 'contratacion' in a.get('href') and a.get('href').endswith('zip')]
periods = [int(link[113:-4]) for link in links]
source_data = {p: l for p, l in zip(periods, links)}

In [4]:
# One file for testing purposes: jan2025
test = 202501
test_zip_url = source_data[test]

In [ ]:
# Get the data of test period: ONLY FOR TESTING PURPOSES
response = requests.get(test_zip_url)

with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
    thezip.extractall(f'data/{test}')

In [5]:
# One file for testing purposes: jan2025
test = 202501
test_zip_url = source_data[test]

# open one file for testing
test_files = os.listdir(f'data/{test}')
test_file = os.path.join(f'data/{test}', test_files[0])

In [6]:
ns, atom, entries = get_atom_data(test_file)
df = get_main_df(entries, ns, mappings)